In [1]:
!pip install seqeval

  Running setup.py bdist_wheel for seqeval ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import os
import json
from seqeval.metrics import f1_score
from sagemaker.tensorflow import TensorFlowPredictor

### Load objects

In [3]:
tag2idx = json.load( open("helpers/objects/tag2idx.json", "r"))
display(tag2idx)

{'B-nat': 0,
 'B-gpe': 1,
 'I-geo': 2,
 'B-org': 3,
 'B-art': 4,
 'I-art': 5,
 'B-per': 6,
 'B-geo': 7,
 'I-eve': 8,
 'I-nat': 9,
 'I-tim': 10,
 'I-org': 11,
 'I-per': 12,
 'O': 13,
 'I-gpe': 14,
 'B-tim': 15,
 'B-eve': 16}

### Load test set

In [4]:
test = pd.read_csv("../data/processed/bilstm_test.csv", encoding='latin', header=None, names=None)
test_y = test.iloc[:,:-50].values
test_X = test.iloc[:,50:].values

### Create predictor from endpoint

In [15]:
predictor = TensorFlowPredictor('tensorflow-training-2020-05-29-19-35-10-455')

In [16]:
predictions = predictor.predict(test_X)
test_pred = np.array(predictions['predictions'])

### Decode tags from test set

In [7]:
idx2tag = {i: w for w, i in tag2idx.items()}

In [17]:
test_labels = []
for i in test_y:
    decoded_sentence = []
    for j in i:
        decoded_sentence.append(idx2tag[j])
    test_labels.append(decoded_sentence)

In [18]:
print(test_labels[1])

['O', 'O', 'B-geo', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### Decode tags from predictions

In [19]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)

In [20]:
print(pred_labels[1])

['O', 'O', 'B-geo', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### Check shapes

In [21]:
assert len(pred_labels) == len(test_labels)
assert len(pred_labels[0]) == len(test_labels[0])

### Compute score

In [22]:
test_f1 = f1_score(pred_labels, test_labels)
print(f"Test F1-Score: {test_f1}")

Test F1-Score: 0.7851865200937106
